In [1]:
import pandas as pd
import numpy as np

np.random.seed(2021)

train_data = pd.read_csv("/content/drive/MyDrive/캐글/santander/santander-product-recommendation/train_ver2.csv/train_ver2.csv")
test_data = pd.read_csv("/content/drive/MyDrive/캐글/santander/santander-product-recommendation/test_ver2.csv/test_ver2.csv")

prods = train_data.columns[24:].tolist()
train_data[prods] = train_data[prods].fillna(0.0).astype(np.int8)

# 제품을 하나도 보유하지 않는 고객 데이터 제거
no_product = train_data[prods].sum(axis=1) == 0
train_data = train_data[~no_product]

for col in train_data.columns[24:]:
  test_data[col] = 0
df = pd.concat([train_data,test_data],axis=0)

features = []

# 범주형 데이터 컬럼
categorical_cols = ["ind_empleado","pais_residencia",'sexo','tiprel_1mes','indresi','indext','conyuemp','canal_entrada','indfall','tipodom','nomprov','segmento']

# 범주형 데이터는 factorize를 통해 label encoding
for col in categorical_cols:
  df[col], _ = df[col].factorize(na_sentinel=-99)
features += categorical_cols

# 수치 변수의 결측값은 -99로 설정
df['age'].replace(' NA',-99,inplace=True)
df['age'] = df['age'].astype(np.int8)

df['antiguedad'].replace('     NA',-99,inplace=True)
df['antiguedad'] = df['antiguedad'].astype(np.int8)

df['renta'].replace('         NA',-99,inplace=True)
df['renta'].fillna(-99, inplace=True)
df['renta'] = df['renta'].astype(float).astype(np.int8)

df['indrel_1mes'].replace('P',-99,inplace=True)
df['indrel_1mes'].fillna(-99,inplace=True)
df['indrel_1mes'] = df['indrel_1mes'].astype(float).astype(np.int8)

# 학습에 이용할 수치형 변수 
features += ['age','antiguedad','renta','ind_nuevo','indrel','indrel_1mes','ind_actividad_cliente']


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,8,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# 첫 계약을 맺은 날짜와, 마지막으로 1등급이었던 날짜로부터 연도, 월 정보 추출
df["fecha_alta_month"] = df["fecha_alta"].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df["fecha_alta_year"] = df["fecha_alta"].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)

df["ult_fec_cli_1t_month"] = df["ult_fec_cli_1t"].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df["ult_fec_cli_1t_year"] = df["ult_fec_cli_1t"].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)

features += ["fecha_alta_month","fecha_alta_year","ult_fec_cli_1t_month","ult_fec_cli_1t_year"]

df.fillna(-99,inplace=True)

# lag-1데이터 생성
def date_to_int(str_date):
  Y, M, D = [int(a) for a in str_date.strip().split("-")]
  int_date = (int(Y)-2015) * 12 + int(M)
  return int_date

df["int_date"] = df["fecha_dato"].map(date_to_int).astype(np.int8)

df_lag = df.copy()
df_lag.columns = [col + "_prev" if col not in ['ncodpers','int_date'] else col for col in df.columns]
df_lag['int_date'] += 1

df_trn = df.merge(df_lag, on=['ncodpers','int_date'],how='left')

# 메모리 효율을 위해 삭제
del df, df_lag

# 저번 달의 제품 정보가 존재하지 않으면 0으로 대체
for prod in prods:
  prev = prod + "_prev"
  df_trn[prev].fillna(0, inplace=True)
df_trn.fillna(-99,inplace=True)

features += [feature + "_prev" for feature in features]
features += [prod + "_prev" for prod in prods]
